# Приоритизация фичей бэклога методами RICE и ICE
подготовил: [Вова Сизов](mailto:vladimirsizov@yandex.ru)

<hr>
Вы получили данные от отдела маркетинга с просьбой провести оценку и приоритизацию фичей продуктового бэклога.  
Также для проведения оценки вам предоставили данные о текущих продажах на основе которых необходимо рассчитать метрики.

Задача:
- Рассчитать продуктовые метрики на данных текущих продаж.    
- Проверить размер плеча метрик с увеличением CM на 25%, опредилить метрики с максимальным плечом.
- Для каждой фичи бэклога рассчитать impact (как размер прироста CM), учитывая указанные параметры изменения метрик каждой фичи (metric - метрика которая должна измениться, change_size_metric - размер ожидаемого изменения) в случае ожидаемого успеха при её внедрении.  
- С помощью методов RICE и ICE приоритизировать фичи бэклога.
- Записать результат в файл excel.

Метрики для расчёта:  
CM (contribution margin) - маржинальная прибыль  
CR1 (conversion rate) - конверсия в первую покупку  
Customers - количество покупателей  
APC (average payment count) - повторные покупки  
Orders - количество заказов  
AvgCheck (average check) - средний чек  
Revenue - валовая прибыль  
CPAcq (cost per acquision) - стоимость привлечения пользователя  
AMPU (average margin per user) средний доход с пользователя  
AC (acquisition costs) = стоимость привлечения (всего)  

Margin - коэффециент валовой прибыли равен 0.2

<hr>
<a id="0"></a>

## Решение

1. [Проверка качества и подготовка данных](#1)
2. [Расчёт продуктовых метрик](#2)
3. [Расчёт RICE и ICE](#3)
4. [Заключение](#4)

<hr>

<a id="1"></a>
# Проверка качества и подготовка данных
[в оглавление](#0)  


In [43]:
import inspect

import pandas as pd
# снимаем ограничение на ширину столбцов
pd.set_option('display.max_colwidth', False)

### Данные:
hypotesis - таблица с фичами 

devices - склейка устройств и User ID   
installs данные регистрации в системе, и источниках трафика  
events - данные об активности просмотра товаров  
checks - данные о покупках и их стоимости  

In [2]:
# получим данные
hypothesis = pd.read_excel('hypothesis.xlsx')

devices = pd.read_csv('devices.csv', dtype={'DeviceID': 'uint64', 'UserID': 'uint64'})
installs = pd.read_csv('installs.csv', parse_dates=['InstallationDate'])
events = pd.read_csv('events.csv', parse_dates=['EventDate'])
revenues = pd.read_csv('revenues.csv', parse_dates=['BuyDate'])

<hr>

### Список гипотез  

Hypothesis — краткое описание гипотезы  
Reach — охват пользователей  в долях  
Impact — размер увеличения GM  
confidence — уверенность в гипотезе по 10-балльной шкале  
effort — затраты ресурсов на проверку гипотезы по 10-балльной шкале  

Для каждой таблицы:
- проверим содержимое  
- информацию о типах и количестве записей  
- наличие пропусков  
- наличие полных дубликатов  

In [3]:
# обзор данных
def data_check(data):
    display(data.head())
    display(data.info())
    null_values = data.isnull().sum().max()
    if null_values == 0:
        print('пропуски:', data.isnull().sum().max())
    else:
        print('пропуски:', data.isnull().sum())
    print('дубликаты:', data.duplicated().sum())

In [4]:
# таблица hypothesis
data_check(hypothesis)

,hypothesis,metric,change_size_metric,reach,impact,confidence,effort
0,"добавить кнопку, ""мой размер"", фильтр отображения товаров только моего размера, множественный выбор размера\nувеличит конверсию на 5%",cr1,0.05,0.85,NaN,5,5
1,"общие рекомендации по изображениям: подвижные живые люди (улыбаются посетителю сайта) в реальных ситуациях, на улице, на море, в спорт клубе, мчатся на машине в путешествие и тд\nувеличит количество повторных покупок на 14%",apc,0.14,1.00,NaN,7,8
2,"главная страница, меню, кнопка ♡ - сылка страница понравившиеся товары\nувеличит средний чек на 2%",avg_check,0.02,1.00,NaN,3,2
3,"главная страница, баннер-слайдер с самыми триггерными предложениями: ""покупки к новому сезону"", ""самый продаваемый или хайповый товар"", и тд\nживые фото товара на человеке в жизни, либо студийное в антуражном пространстве (не на белом фоне) добавляет свежести сайту\nувеличит конверсию на 6%",cr1,0.06,1.00,NaN,6,6
4,"главная страница, навигационный слайдер основные категории товаров блоками текст и картинка.\nИзменить стиль картинки на товара целиком / несколько товаров / товар на человеке\nувеличит конверсию на 3%",cr1,0.03,1.00,NaN,6,5


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   hypothesis          15 non-null     object 
 1   metric              15 non-null     object 
 2   change_size_metric  15 non-null     float64
 3   reach               15 non-null     float64
 4   impact              0 non-null      float64
 5   confidence          15 non-null     int64  
 6   effort              15 non-null     int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 968.0+ bytes


None

пропуски: hypothesis            0 
metric                0 
change_size_metric    0 
reach                 0 
impact                15
confidence            0 
effort                0 
dtype: int64
дубликаты: 0


<hr>

### Склейка устройств и User ID

Особенность сервиса заключается в том, что для просмотра товаров не нужна авторизация.  
До момента авторизации про пользователя известен только его DeviceID — идентификатор устройства.  
При этом для совершения покупки логин обязателен.  
На моменте авторизации пользователю присваивается UserID, и тогда мы уже знаем два его идентификатора: DeviceID (устройство) и UserID (логин).  
Так как на этапах установки приложения и просмотра каталога пользователь еще может быть не авторизован, там мы сохраняем только DeviceID.  
Но так как покупки нельзя совершить без авторизации, то покупки сохраняются только с UserID.  
Для того чтобы просмотры и установки можно было объединить с покупками, нам нужна таблица соответствия DeviceID к UserID, то есть таблица devices:

DeviceID — идентификатор устройства  
UserID — идентификатор пользователя

In [5]:
data_check(devices)

,DeviceID,UserID
0,2061461442082440457,5890312570997226086
1,2061502319468333809,7597630269297929747
2,2061549705656161924,4684956878445305718
3,2061590374665983626,4564106340145658170
4,2061630421844171981,16634945760702342388


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470332 entries, 0 to 470331
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   DeviceID  470332 non-null  uint64
 1   UserID    470332 non-null  uint64
dtypes: uint64(2)
memory usage: 7.2 MB


None

пропуски: 0
дубликаты: 0


<hr>

### Данные о регистрации в системе

DeviceID — идентификатор устройства  
InstallationDate — дата регистрации  
InstallCost — цена привлечения клиента  
Platform — платформа  
Source — источник трафика  

In [6]:
data_check(installs)

,DeviceID,InstallationDate,InstallCost,Platform,Source
0,2061461442082440457,2019-02-19,1.33,android,Source_14
1,2061549705656161924,2019-02-18,0.00,android,Source_27
2,2061590374665983626,2019-01-24,0.00,iOS,Source_27
3,2061630421844171981,2019-09-03,0.00,android,Source_27
4,2061656490978911882,2019-04-22,1.12,android,Source_14


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286154 entries, 0 to 286153
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   DeviceID          286154 non-null  uint64        
 1   InstallationDate  286154 non-null  datetime64[ns]
 2   InstallCost       286154 non-null  float64       
 3   Platform          286154 non-null  object        
 4   Source            286154 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(2), uint64(1)
memory usage: 10.9+ MB


None

пропуски: 0
дубликаты: 0


<hr>

### Данные об активности просмотра товаров

DeviceID — идентификатор устройства  
AppPlatform — платформа  
EventDate — дата события  
events — количество просмотров всех товаров за этот день у этого DeviceID.  

In [7]:
data_check(events)

,DeviceID,AppPlatform,events,EventDate
0,10116097407821560691,android,51,2019-01-04
1,10116141036402249962,android,1,2019-01-04
2,10116163208905002180,android,0,2019-01-05
3,10116163208905002180,android,53,2019-01-07
4,10116163208905002180,android,8,2019-01-09


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8550265 entries, 0 to 8550264
Data columns (total 4 columns):
 #   Column       Dtype         
---  ------       -----         
 0   DeviceID     uint64        
 1   AppPlatform  object        
 2   events       int64         
 3   EventDate    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1), uint64(1)
memory usage: 260.9+ MB


None

пропуски: 0
дубликаты: 0


<hr>

### Данные о покупках  

UserID — идентификатор пользователя   
Revenue — чек пользователя  
BuyDate — дата  

In [8]:
data_check(revenues)

,UserID,Revenue,BuyDate
0,5890312570997226086,11,2019-02-25
1,5890312570997226086,3,2019-02-22
2,5890312570997226086,1,2019-02-21
3,4684956878445305718,5,2019-03-29
4,4684956878445305718,8,2019-03-22


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120651 entries, 0 to 120650
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   UserID   120651 non-null  uint64        
 1   Revenue  120651 non-null  int64         
 2   BuyDate  120651 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), uint64(1)
memory usage: 2.8 MB


None

пропуски: 0
дубликаты: 0


### Преобразование данных

In [9]:
# скорректируем типы данных у таблицы hypothesis
hypothesis['hypothesis'] = hypothesis.hypothesis.astype('string')
hypothesis['metric'] = hypothesis.metric.astype('string')

In [10]:
# избавимся от UserID в revenue чтобы работать только с DeviceID
revenues = revenues.merge(devices, on='UserID', how='left').drop(columns='UserID')
revenues.head()

,Revenue,BuyDate,DeviceID
0,11,2019-02-25,2061461442082440457
1,3,2019-02-22,2061461442082440457
2,1,2019-02-21,2061461442082440457
3,5,2019-03-29,2061549705656161924
4,8,2019-03-22,2061549705656161924


In [11]:
# для удобства работы приведём название колонок у таблиц к нижнему регистру
revenues = revenues.rename(columns=lambda c: c.lower())
installs = installs.rename(columns=lambda c: c.lower())
events = events.rename(columns=lambda c: c.lower())

<hr>

<a id="2"></a>
# Расчёт продуктовых метрик
[в оглавление](#0)  



Для расчёта метрик нам понадобятся несколько функций.

In [12]:
def get_cm(revenue=False, margin=False, ac=False):
    """посчитать CM (contribution margin) - маржинальная прибыль"""
    if revenue and margin and ac:
        return revenue * margin - ac
    else:
        print('Для вычисления CM не достаточно метрик.')

In [13]:
def get_cr1(customers=False, users=False):
    """посчитать CR1 (conversion rate) конверсия в первую покупку"""
    if customers and users:
         return customers / users
    else:
        print('Для вычисления CR1 не достаточно метрик.')

In [14]:
def get_customers(users=False, cr1=False):
    """посчитать Customers  - количество покупателей"""
    if users and cr1:
         return users * cr1
    else:
        print('Для вычисления Customers не достаточно метрик.')

In [15]:
def get_apc(orders=False, customers=False):
    """посчитать APC (average payment count) - повторные покупки"""
    if orders and customers:
         return orders / customers
    else:
        print('Для вычисления APC не достаточно метрик.')

In [16]:
def get_orders(apc=False, customers=False):
    """посчитать Orders - количество заказов"""
    if apc and customers:
         return apc * customers
    else:
        print('Для вычисления Orders не достаточно метрик.')

In [17]:
def get_avg_check(revenue=False, orders=False):
    """посчитать AvgCheck (average check) - средний чек"""
    # более верно считать средний чек следует как среднее значение среднего чека каждого пользователя
    # но в рамках данного эксперимента мы будем пользоваться упрощенной схемой
    if revenue and orders:
         return revenue / orders
    else:
        print('Для вычисления AvgCheck не достаточно метрик.')

In [18]:
def get_revenue(avg_check=False, orders=False):
    """посчитать Revenue - валовая прибыль"""
    if avg_check and orders:
         return avg_check * orders
    else:
        print('Для вычисления Orders не достаточно метрик.')

In [19]:
def get_cpa_cq(ac=False, users=False):
    """посчитать CPAcq (cost per acquision) - стоимость привлечения пользователя"""
    if ac and users:
         return ac / users
    else:
        print('Для вычисления CPAcq не достаточно метрик.')

In [20]:
def get_ampu(revenue=False, margin=False, users=False):
    """посчитать AMPU (average margin per user) средний доход с пользователя"""
    if revenue and margin and users:
         return revenue * margin / users
    else:
        print('Для вычисления AMPU не достаточно метрик.')

In [21]:
def get_ac(cpa_cq=False, users=False):
    """посчитать AC (acquisition costs) = стоимость привлечения (всего)"""
    if cpa_cq and users:
         return cpa_cq * users
    else:
        print('Для вычисления AMPU не достаточно метрик.')

In [27]:
def get_metrics(cm=False,
                users=False,
                cr1=False,
                customers=False,
                apc=False,
                orders=False,
                avg_check=False,
                revenue=False,
                margin=False,
                cpa_cq=False,
                ampu=False,
                ac=False):
    """посчитать метрики"""
    metrics = {'users': users, 
               'cr1': cr1,
               'customers': customers,
               'apc': apc,
               'orders': orders,
               'avg_check': avg_check,
               'revenue': revenue,
               'margin': margin,
               'cpa_cq': cpa_cq,
               'ampu': ampu,
               'ac': ac,
               'cm': cm}
    # собираем недостющие метрики
    while False in metrics.values():
        for key, value in metrics.items():
            # если метрики нет
            if value == False:
                name_function = 'get_' + key
                # пытаемся посчитать отсутствующую метрику
                try:
                    # получаем список аргументов функции которую хотим вызвать
                    args = inspect.getfullargspec(globals()[name_function]).args
                    # собираем имена аргументов функции которую хотим вызвать
                    values_args = {}
                    for i in args:
                        # если значение аргумента(в метриках) есть
                        if metrics[i]:
                            values_args[i] = metrics[i]
                    # вызываем функцию расчета значения недостающей ментрики
                    result = globals()[name_function](**values_args)
                    # добавляем новую метрику к имеющимся
                    if result:
                        metrics[key] = result
                except:
                    pass
    return metrics

In [23]:
def get_diff_cm(correct_cr1=False,
                correct_users=False,
                correct_apc=False,
                correct_margin=False,
                correct_avg_check=False,
                correct_ac=False,
                show='off',
                data=False):
    """Получить разницу cm относительно текущего показателя"""
    # считаем метрики с учётом изменения одной из них
    if correct_cr1:
        metrics = get_metrics(users=users, cr1=cr1*correct_cr1, apc=apc,
                              avg_check=avg_check, margin=margin, ac=ac)
        change = 'Изменение CR1*' + str(round(correct_cr1, 2))
    elif correct_users:
        metrics = get_metrics(users=users*correct_users, cr1=cr1, apc=apc,
                            avg_check=avg_check, margin=margin, cpa_cq=cpa_cq)
        change = 'Изменение Users*' + str(round(correct_users, 2))
    elif correct_apc:
        metrics = get_metrics(users=users, customers=customers, apc=apc*correct_apc, 
                          avg_check=avg_check, margin=margin, ac=ac)
        change = 'Изменение APC*' + str(round(correct_apc, 2))
    elif correct_margin:
        metrics = get_metrics(users=users, customers=customers, apc=apc, avg_check=avg_check,
                             margin=margin*correct_margin, cpa_cq=cpa_cq)
        change = 'Изменение Margin*' + str(round(correct_margin, 2))
    elif correct_avg_check:
        metrics = get_metrics(users=users, customers=customers, apc=apc, 
                                avg_check=avg_check*correct_avg_check, margin=margin, ac=ac)
        change = 'Изменение AvgCheck*' + str(round(correct_avg_check, 2))
    elif correct_ac:
        metrics = get_metrics(users=users, customers=customers, apc=apc, avg_check=avg_check,
                          margin=margin, ac=ac*correct_ac)
        change = 'Изменение AC*' + str(round(correct_ac, 2))
    else:
        # считаем базовые показатели
        metrics = get_metrics(users=users, customers=customers, orders=orders, revenue=revenue, ac=ac, margin=margin)
        change = 'Базовые метрики'
    # создаём таблицу метрик
    metrics = pd.DataFrame.from_dict(metrics, orient='index')
    metrics.columns = [change]
    metrics = metrics.T
    # получаем значение изменения CM
    cm = get_metrics(users=users, customers=customers, orders=orders, revenue=revenue, ac=ac, margin=margin)['cm']
    cm_change = round(metrics.cm.values[0] - cm)
    # просим показать таблицу
    if show == 'on':
        display(metrics)
    # возвращаем метрики
    if data:
        # все метрики словарём
        return metrics
    else:
        # только метрику изменения CM
        return cm_change

### Посчитаем базовые метрики из полученых данных о предыдущих продажах
На основании этих метрик мы будем строить расчёты плеч метрик и приоретизировать гипотезы

In [24]:
# считаем метрики из исходных данных
# Users - количество пользователей
users = len(devices)
# Customers  - количество покупателей
customers = revenues.deviceid.nunique()
# Orders - количество заказов
orders = len(revenues)
# Revenue - валовая прибыль
revenue = revenues.revenue.sum()
# AC (acquisition costs) = стоимость привлечения (всего)
ac = installs.installcost.sum()

# Margin - коэффециент валовой прибыли
margin = 0.2

In [28]:
# расчитываем остальные базовые метрики
base_metrics = get_diff_cm(data=True)

# CR1 (conversion rate) конверсия в первую покупку
cr1 = base_metrics['cr1'][0]
# APC (average payment count) - повторные покупки
apc = base_metrics['apc'][0]
# AvgCheck (average check) - средний чек
avg_check = base_metrics['avg_check'][0]
# CPAcq (cost per acquision) - стоимость привлечения пользователя
cpa_cq = base_metrics['cpa_cq'][0]
# AMPU (average margin per user) средний доход с пользователя
ampu = base_metrics['ampu'][0]
# CM (contribution margin) - маржинальная прибыль
cm = base_metrics['cm'][0]


### Посмотрим на плечи метрик при изменении каждой из них
Первая таблица это базовые расчеты метрик - исходные данные без изменений.

In [29]:
# пример считаем базовые метрики (без изменений в метриках)
get_diff_cm(show='on')

,users,cr1,customers,apc,orders,avg_check,revenue,margin,cpa_cq,ampu,ac,cm
Базовые метрики,470332.0,0.03991,18771.0,6.427521,120651.0,13.273931,1601513.0,0.2,0.439074,0.681014,206510.67,113791.93


0

В примерах ниже мы рассчитываем увеличение CM на 25% (+27852)

In [30]:
# пример считаем метрики с изменённым cr1
get_diff_cm(correct_cr1=1.0869566, show='on')

,users,cr1,customers,apc,orders,avg_check,revenue,margin,cpa_cq,ampu,ac,cm
Изменение CR1*1.09,470332.0,0.043381,20403.262339,6.427521,131142.400747,13.273931,1.740775e+06,0.2,0.439074,0.740232,206510.67,141644.355067


27852

In [31]:
# пример считаем метрики с изменённым users
get_diff_cm(correct_users=1.24476, show='on')

,users,cr1,customers,apc,orders,avg_check,revenue,margin,cpa_cq,ampu,ac,cm
Изменение Users*1.24,585450.46032,0.03991,23365.38996,6.427521,150181.53876,13.273931,1.993499e+06,0.2,0.439074,0.681014,257056.221589,141643.642787


27852

In [32]:
# пример считаем метрики с изменённым apc
get_diff_cm(correct_apc=1.0869566, show='on')

,users,cr1,customers,apc,orders,avg_check,revenue,margin,cpa_cq,ampu,ac,cm
Изменение APC*1.09,470332.0,0.03991,18771.0,6.986437,131142.400747,13.273931,1.740775e+06,0.2,0.439074,0.740232,206510.67,141644.355067


27852

In [33]:
# пример считаем метрики с изменённым margin
get_diff_cm(correct_margin=1.0869566, show='on')

,users,cr1,customers,apc,orders,avg_check,revenue,margin,cpa_cq,ampu,ac,cm
Изменение Margin*1.09,470332.0,0.03991,18771.0,6.427521,120651.0,13.273931,1601513.0,0.217391,0.439074,0.740232,206510.67,141644.355067


27852

In [34]:
# пример считаем метрики с изменённым avg_check
get_diff_cm(correct_avg_check=1.0869566, show='on')

,users,cr1,customers,apc,orders,avg_check,revenue,margin,cpa_cq,ampu,ac,cm
Изменение AvgCheck*1.09,470332.0,0.03991,18771.0,6.427521,120651.0,14.428186,1.740775e+06,0.2,0.439074,0.740232,206510.67,141644.355067


27852

In [35]:
# пример считаем метрики с изменённым ac
get_diff_cm(correct_ac=0.86513, show='on')

,users,cr1,customers,apc,orders,avg_check,revenue,margin,cpa_cq,ampu,ac,cm
Изменение AC*0.87,470332.0,0.03991,18771.0,6.427521,120651.0,13.273931,1601513.0,0.2,0.379856,0.681014,178658.575937,141644.024063


27852

Максимальное плечо (наибольшее влияние при наименьшем изменениии) у метрик: cr1, apc, margin, avg_check

<hr>

<a id="3"></a>
# Расчёт RICE и ICE
[в оглавление](#0)  



Задача RICE и ICE методов заключается в приоритезации фич в продуктовом бэклоге на основании фактических данных. 

- Сперва мы рассчитаем для таблицы hypothesis метрику impact  
- Затем приоритезируем методами RICE и ICE  
- В конце запишем результаты в фаил excel  

In [36]:
def get_impact(metric, change_size_metric):
    """расчёт колонки impact"""
    if metric == 'cr1':
        result = get_diff_cm(correct_cr1=1+change_size_metric)
    elif metric == 'users':
        result = get_diff_cm(correct_users=1+change_size_metric)
    elif metric == 'apc':
        result = get_diff_cm(correct_apc=1+change_size_metric)
    elif metric == 'margin':
        result = get_diff_cm(correct_margin=1+change_size_metric)
    elif metric == 'avg_check':
        result = get_diff_cm(correct_avg_check=1+change_size_metric)
    elif metric == 'ac':
        result = get_diff_cm(correct_ac=1+change_size_metric)
    return result

Метрика impact представляет из себя размер фактической дополнительной прибыли CM которую бы мы гипотетически получили при внедрении фичи.  
Данные о прибыли будем рассчитывать из показателей заявленых в таблице hypothesis, на основании данных предыдущих продаж, а также полученых формул расчёта метрик.

Для расчета impact служат колонки:
- метрика на которую повлияет фича (metric)
- размер изменения (change_size_metric).

Рассчитаем impact для каждой фичи в таблице hypothesis:

In [37]:
# Рассчитаем impact для каждой фичи в таблице hypothesis
hypothesis['impact'] = hypothesis.apply(lambda x:
                        get_impact(metric=x.metric, change_size_metric=x.change_size_metric), axis=1)
hypothesis.head()

,hypothesis,metric,change_size_metric,reach,impact,confidence,effort
0,"добавить кнопку, ""мой размер"", фильтр отображения товаров только моего размера, множественный выбор размера увеличит конверсию на 5%",cr1,0.05,0.85,16015,5,5
1,"общие рекомендации по изображениям: подвижные живые люди (улыбаются посетителю сайта) в реальных ситуациях, на улице, на море, в спорт клубе, мчатся на машине в путешествие и тд увеличит количество повторных покупок на 14%",apc,0.14,1.00,44842,7,8
2,"главная страница, меню, кнопка ♡ - сылка страница понравившиеся товары увеличит средний чек на 2%",avg_check,0.02,1.00,6406,3,2
3,"главная страница, баннер-слайдер с самыми триггерными предложениями: ""покупки к новому сезону"", ""самый продаваемый или хайповый товар"", и тд живые фото товара на человеке в жизни, либо студийное в антуражном пространстве (не на белом фоне) добавляет свежести сайту увеличит конверсию на 6%",cr1,0.06,1.00,19218,6,6
4,"главная страница, навигационный слайдер основные категории товаров блоками текст и картинка. Изменить стиль картинки на товара целиком / несколько товаров / товар на человеке увеличит конверсию на 3%",cr1,0.03,1.00,9609,6,5


Теперь для каждой фичи посчитаем RICE по формуле:  
    RICE = reach * impact * confidence / effort  
    
Также приоритезируем фичи в продуктовом бэклоге (RICE_priority)

In [38]:
# посчитаем RICE
rice = hypothesis.copy()
rice['RICE'] = round((rice.reach * rice.impact * rice.confidence) / rice.effort)
rice = rice.sort_values('RICE', ascending=False).reset_index()
rice['RICE_priority'] = rice.index + 1
rice = rice.set_index('index')
rice.head()

,hypothesis,metric,change_size_metric,reach,impact,confidence,effort,RICE,RICE_priority
index,,,,,,,,,
12,"карточка товара, блок фото с панелькой превью заменить на: полноценную галерею всех изображений товара под разными ракурсами увеличит количество повторных покупок на 14%",apc,0.14,1.00,44842,8,4,89684.0,1
13,"карточка товара, первое из изображений - видео 360 градусов увеличит количество повторных покупок на 11%",apc,0.11,1.00,35233,9,6,52850.0,2
10,"страница категории товаров, внизу под блоком с плитками товаров, добавить блок с сылками на 3 статьи с FAQ по категории (для удержания пользователя если он сомневается в выборе или покупке), например ""как ухаживать за сумкой?"", или ""правильно подбираем размер толстовки"" увеличит конверсию на 13%",cr1,0.13,0.45,41639,5,2,46844.0,3
1,"общие рекомендации по изображениям: подвижные живые люди (улыбаются посетителю сайта) в реальных ситуациях, на улице, на море, в спорт клубе, мчатся на машине в путешествие и тд увеличит количество повторных покупок на 14%",apc,0.14,1.00,44842,7,8,39237.0,4
5,"главная страница, трехслойный слайдер с превью товарных карточек, по категориям: ""новые поступления"", ""какие тренды?"", ""пользователи выбирают"" увеличит количество повторных покупок на 9%",apc,0.09,1.00,28827,6,6,28827.0,5


Теперь для каждой фичи посчитаем ICE по формуле:  
    ICE = reach * impact * easy  
    
Также приоритезируем фичи в продуктовом бэклоге (ICE_priority)

In [39]:
# считаем ICE
ice = hypothesis.copy()
ice['easy'] = 10 - ice.effort
ice.drop(columns='effort', inplace=True)
ice['ICE'] = round(ice.reach * ice.impact * ice.easy)
ice = ice.sort_values('ICE', ascending=False).reset_index()
ice['ICE_priority'] = ice.index + 1
ice = ice.set_index('index')
ice.head()

,hypothesis,metric,change_size_metric,reach,impact,confidence,easy,ICE,ICE_priority
index,,,,,,,,,
12,"карточка товара, блок фото с панелькой превью заменить на: полноценную галерею всех изображений товара под разными ракурсами увеличит количество повторных покупок на 14%",apc,0.14,1.00,44842,8,6,269052.0,1
10,"страница категории товаров, внизу под блоком с плитками товаров, добавить блок с сылками на 3 статьи с FAQ по категории (для удержания пользователя если он сомневается в выборе или покупке), например ""как ухаживать за сумкой?"", или ""правильно подбираем размер толстовки"" увеличит конверсию на 13%",cr1,0.13,0.45,41639,5,8,149900.0,2
13,"карточка товара, первое из изображений - видео 360 градусов увеличит количество повторных покупок на 11%",apc,0.11,1.00,35233,9,4,140932.0,3
11,"страница категории товаров, выше футера - текстовый блок с категориями товаров увеличит количество повторных покупок на 7%",apc,0.07,1.00,22421,4,6,134526.0,4
5,"главная страница, трехслойный слайдер с превью товарных карточек, по категориям: ""новые поступления"", ""какие тренды?"", ""пользователи выбирают"" увеличит количество повторных покупок на 9%",apc,0.09,1.00,28827,6,4,115308.0,5


Наконец обьединим и отсортируем данные RICE_priority и ICE_priority

In [40]:
excel = hypothesis.merge(ice[['hypothesis', 'easy', 'ICE_priority']], on='hypothesis') \
    .merge(rice[['hypothesis', 'RICE_priority']], on='hypothesis').sort_values('RICE_priority')
excel

,hypothesis,metric,change_size_metric,reach,impact,confidence,effort,easy,ICE_priority,RICE_priority
12,"карточка товара, блок фото с панелькой превью заменить на: полноценную галерею всех изображений товара под разными ракурсами увеличит количество повторных покупок на 14%",apc,0.140,1.00,44842,8,4,6,1,1
13,"карточка товара, первое из изображений - видео 360 градусов увеличит количество повторных покупок на 11%",apc,0.110,1.00,35233,9,6,4,3,2
10,"страница категории товаров, внизу под блоком с плитками товаров, добавить блок с сылками на 3 статьи с FAQ по категории (для удержания пользователя если он сомневается в выборе или покупке), например ""как ухаживать за сумкой?"", или ""правильно подбираем размер толстовки"" увеличит конверсию на 13%",cr1,0.130,0.45,41639,5,2,8,2,3
1,"общие рекомендации по изображениям: подвижные живые люди (улыбаются посетителю сайта) в реальных ситуациях, на улице, на море, в спорт клубе, мчатся на машине в путешествие и тд увеличит количество повторных покупок на 14%",apc,0.140,1.00,44842,7,8,2,7,4
5,"главная страница, трехслойный слайдер с превью товарных карточек, по категориям: ""новые поступления"", ""какие тренды?"", ""пользователи выбирают"" увеличит количество повторных покупок на 9%",apc,0.090,1.00,28827,6,6,4,5,5
9,"страница категории товаров, внизу под блоком с плитками товаров, добавить блок с описанием категории (общая расширенная статья, описывающая какие класные, стильные штуки продаются в категории, какие крутые люди и сколько времени потратили на создание этих модных и чудесных вещей) увеличит средний чек на 4%",avg_check,0.040,1.00,12812,2,1,9,6,6
8,"страница категории товаров, в превью товарной карточки, в текстовое описание добавить текст например ""4 цвета"", для товара у которого есть смежные карточки. При наведении - всплывающее окно с миниатюрами товаров в разных цветах, избавляет от однообразия в категории увеличит средний чек на 8,7%",avg_check,0.087,1.00,27866,7,8,2,11,7
11,"страница категории товаров, выше футера - текстовый блок с категориями товаров увеличит количество повторных покупок на 7%",apc,0.070,1.00,22421,4,4,6,4,8
3,"главная страница, баннер-слайдер с самыми триггерными предложениями: ""покупки к новому сезону"", ""самый продаваемый или хайповый товар"", и тд живые фото товара на человеке в жизни, либо студийное в антуражном пространстве (не на белом фоне) добавляет свежести сайту увеличит конверсию на 6%",cr1,0.060,1.00,19218,6,6,4,8,9
0,"добавить кнопку, ""мой размер"", фильтр отображения товаров только моего размера, множественный выбор размера увеличит конверсию на 5%",cr1,0.050,0.85,16015,5,5,5,9,10


In [41]:
# записываем результаты в файл
excel.to_excel("hypothesis_priority_RICE_ICE.xlsx", index=False)